In [2]:
from satellite_analysis.graphs import massrelation as massrelation
from satellite_analysis.massrelations import stellarmass_relations
import yt
import glob
import numpy as np
import matplotlib.pyplot as plt


#first we define the plotting function that will be used at the end

def centralandsatellitegalaxypapercomparisons(gas_mass_central, gas_ratio_central, star_mass_central, star_ratio_central,
                                              baryon_mass_central, baryon_ratio_central, central_all_mass_mvir,
                                              satellite_all_mass_mvir, output_dir, a, x, y1, y2):
    plt.figure(1, figsize=(20,30))
    
    plt.subplot(321)
    plt.title('Scale Factor %s' % str(a/1000))
    plt.xlabel('Halo Mass')
    plt.ylabel('Gas Mass / Halo Mass')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    #plt.ylim(0.0, 0.10)
    plt.plot(x, y1)
    plt.scatter(central_all_mass_mvir, gas_ratio_central, s = 1)
    #plt.scatter(satellite_all_mass_mvir, gas_ratio_satellite, s = 1, c = 'r')
    plt.legend(('Omega Baryon / Omega Matter = 0.167',  'Host Galaxies'))
    
    plt.subplot(322)
    plt.xlabel('Halo Mass (Darkmatter + Stars + Gas @ Rvir)')
    plt.ylabel('Gas Mass (Gas @ Rvir)')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    plt.ylim(10**(-2), 10**(14))
    plt.plot(x, y2)
    plt.scatter(central_all_mass_mvir, gas_mass_central, s = 1)
    
    plt.subplot(323)
    plt.title('Scale Factor %s' % str(a/1000))
    plt.xlabel('Halo Mass')
    plt.ylabel('Stellar Mass / Halo Mass')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    plt.ylim(1.0, 10**-6)
    plt.plot(x, y1)
    plt.scatter(central_all_mass_mvir, star_ratio_central, s = 1)
    #plt.scatter(satellite_all_mass_mvir, gas_ratio_satellite, s = 1, c = 'r')
    plt.legend(('Omega Baryon / Omega Matter = 0.167',  'Host Galaxies'))
    
    plt.subplot(324)
    plt.xlabel('Halo Mass (Darkmatter + Stars + Gas @ Rvir)')
    plt.ylabel('Stellar Mass (Stars @ Rvir)')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    plt.ylim(10**(-2), 10**(14))
    plt.plot(x, y2)
    plt.scatter(central_all_mass_mvir, star_mass_central, s = 1)
    
    plt.subplot(325)
    plt.title('Scale Factor %s' % str(a/1000))
    plt.xlabel('Halo Mass')
    plt.ylabel('Baryon Mass / Halo Mass')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    #plt.ylim(0.0, 0.10)
    plt.plot(x, y1)
    plt.scatter(central_all_mass_mvir, baryon_ratio_central, s = 1)
    #plt.scatter(satellite_all_mass_mvir, gas_ratio_satellite, s = 1, c = 'r')
    plt.legend(('Omega Baryon / Omega Matter = 0.167',  'Host Galaxies'))
    
    plt.subplot(326)
    plt.xlabel('Halo Mass (Darkmatter + Stars + Gas @ Rvir)')
    plt.ylabel('Baryon Mass (Star + Gas @ Rvir)')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    plt.ylim(10**(-2), 10**(14))
    plt.plot(x, y2)
    plt.scatter(central_all_mass_mvir, baryon_mass_central, s = 1)
    #plt.scatter(satellite_all_mass_mvir, gas_mass_satellite, s = 1, c = 'r')
    plt.savefig('%s/baryonratios%s.png' % (output_dir, str(a)))
    plt.close()


#first find all of the halomass catalogs

#add argparse and make the out_dir if it doesnt exits
output_dir = '/Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/gasratio'
halomass_dirs = glob.glob('/Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA*')


#now, find the earliest and latest snapshots for each catalog
earliest_snap = []
latest_snap = []
for directory in halomass_dirs:
    all_snaps = glob.glob('%s/*.ascii' % directory)
    all_snaps.sort()
    earliest_snap.append(all_snaps[0])
    latest_snap.append(all_snaps[-1])

earliest_snap = earliest_snap[0][-9:-6]
latest_snap = latest_snap[-1][-9:-6]

#now make the list of all possible snapshots from the earlies and latest snap
a_list = [int(earliest_snap) + 10*x for x in range((int(latest_snap) - int(earliest_snap))//10 + 1)]

#now now I set up a nested loop to iterate over each a in my a_list in each of my VELA_dir halomass folder
for a in a_list:
    print('Generating dataset for a: %s' % a)
    
    central_ids, satellite_ids = [], []
    
    gas_mass_central, gas_mass_satellite = [], []
    
    star_mass_central, star_ratio_central = [], []
    
    baryon_mass_central, baryon_ratio_central = [], []
    
    gas_ratio_central, gas_ratio_satellite = [], []
    
    central_all_mass_mvir, satellite_all_mass_mvir = [], []
    
    for halomass_dir in halomass_dirs:
        print('Trying file: %s/halomass%s.ascii' % (halomass_dir, a))
        try:
            readfile = open('%s/halomass%s.ascii' % (halomass_dir, a))
            lines = readfile.readlines()
            halomass_lines = []
            for line in lines:
                halomass_lines.append(line.split())
            del halomass_lines[0]
            for data_lists in halomass_lines:
                #filter out the satellites (pid =/= -1) from the central galaxies they orbit (pid = -1)
                if float(data_lists[1]) == -1:
                    central_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    
                    gas_mass_central.append(float(data_lists[8]))
                    
                    star_mass_central.append(float(data_lists[7]))
                    
                    baryon_mass_central.append(float(data_lists[8]) + float(data_lists[7]))
                    
                    #Mvir = darkmatter + stars + gas @ rvir
                    Mvir = float(data_lists[2]) + float(data_lists[7]) + float(data_lists[8])
                    central_all_mass_mvir.append(Mvir)
                    
                    gas_ratio_central.append(float(data_lists[8]) / Mvir)
                    
                    star_ratio_central.append(float(data_lists[7]) / Mvir)
                    
                    baryon_ratio_central.append((float(data_lists[8]) + float(data_lists[7])) / Mvir)
                    
                else:
                    satellite_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    
                    gas_mass_satellite.append(float(data_lists[8]) + float(data_lists[7]))
                    
                    Mpeak = float(data_lists[3]) + float(data_lists[7]) + float(data_lists[8])
                    satellite_all_mass_mvir.append(Mpeak)
                    
                    gas_ratio_satellite.append((float(data_lists[8]) + float(data_lists[7])) / Mpeak)
                    
            readfile.close()
        except FileNotFoundError:
            print('No file: %s/halomass%s.ascii' % (halomass_dir, a))
            
    x = np.linspace(10**7.8, 10**14, 3000)
    y1 = [.167] * 3000
    y2 = [q*.167 for q in x]
    centralandsatellitegalaxypapercomparisons(gas_mass_central, gas_ratio_central, star_mass_central, star_ratio_central,
                                              baryon_mass_central, baryon_ratio_central, central_all_mass_mvir,
                                              satellite_all_mass_mvir, output_dir, a, x, y1, y2)
            
            
            

Generating dataset for a: 100
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass100.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass100.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass100.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA10/

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass160.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass160.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass160.ascii
Generating dataset for a: 170
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass170.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass170.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass170.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass170.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation

No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass230.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass230.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA10/halomass230.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass230.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass230.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass230.ascii
Generating dataset for a: 240
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass240.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VEL

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass280.ascii
Generating dataset for a: 290
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass290.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass290.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass290.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass340.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass340.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass340.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass340.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA10/halomass340.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass340.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass340.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass340.ascii
Generati

No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/halomass390.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA10/halomass390.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass390.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass390.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass390.ascii
Generating dataset for a: 400
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass400.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass400.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VEL

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA10/halomass440.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass440.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass440.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass440.ascii
Generating dataset for a: 450
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass450.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass450.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass450.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VEL

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass490.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass490.ascii
Generating dataset for a: 500
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass500.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass500.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass500.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass500.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass500.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass540.ascii
Generating dataset for a: 550
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass550.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass550.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass550.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass550.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass550.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA08/halomass550.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA09/

Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass590.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA12/halomass590.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA14/halomass590.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA15/halomass590.ascii
Generating dataset for a: 600
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass600.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA06/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/halomass600.ascii
No file: /Users/user1/documents/GEN3/rockstarcomoving001/rockstar_moresnaps/stellarmassrelation/VELA07/

In [5]:
from satellite_analysis.graphs import massrelation as massrelation
from satellite_analysis.massrelations import stellarmass_relations
import yt
import glob
import numpy as np
import matplotlib.pyplot as plt


#first we define the plotting function that will be used at the end

def centralandsatellitegalaxypapercomparisons(gas_mass_central, gas_ratio_central, gas_mass_satellite,
                                              gas_ratio_satellite, central_all_mass_mvir, satellite_all_mass_mvir,
                                              output_dir, a, x, y1, y2):
    plt.figure(1, figsize=(20,10))
    plt.subplot(121)
    plt.title('Scale Factor %s' % str(a/1000))
    plt.xlabel('Halo Mass (Darkmatter + Stars + Gas @ Rvir)')
    plt.ylabel('Gas Mass / Halo Mass')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    #plt.ylim(0.0, 0.10)
    plt.plot(x, y1)
    plt.scatter(central_all_mass_mvir, gas_ratio_central, s = 1)
    plt.scatter(satellite_all_mass_mvir, gas_ratio_satellite, s = 1, c = 'r')
    plt.legend(('Omega Baryon / Omega Matter = 0.16',  'Host Galaxies', 'Satellite Galaxies'))
    
    plt.subplot(122)
    plt.xlabel('Halo Mass')
    plt.ylabel('Gas Mass')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(10**(7.8), 10**(14))
    plt.ylim(10**(-2), 10**(14))
    plt.plot(x, y2)
    plt.scatter(central_all_mass_mvir, gas_mass_central, s = 1)
    plt.scatter(satellite_all_mass_mvir, gas_mass_satellite, s = 1, c = 'r')
    plt.savefig('%s/gasmassratio%s.png' % (output_dir, str(a)))
    plt.close()


#first find all of the halomass catalogs

#add argparse and make the out_dir if it doesnt exits
output_dir = '/Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/gasratio'
halomass_dirs = glob.glob('/Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA*')


central_ids_lists, satellite_ids_lists = [], []

central_sim_lists, satellite_sim_lists = [], []

gas_mass_central_lists, gas_mass_satellite_lists = [], []
    
gas_ratio_central_lists, gas_ratio_satellite_lists = [], []
    
central_all_mass_mvir_lists, satellite_all_mass_mvir_lists = [], []



#now, find the earliest and latest snapshots for each catalog
earliest_snap = []
latest_snap = []
for directory in halomass_dirs:
    all_snaps = glob.glob('%s/*.ascii' % directory)
    all_snaps.sort()
    earliest_snap.append(all_snaps[0])
    latest_snap.append(all_snaps[-1])

earliest_snap = earliest_snap[0][-9:-6]
latest_snap = latest_snap[-1][-9:-6]

#now make the list of all possible snapshots from the earlies and latest snap
a_list = [int(earliest_snap) + 10*x for x in range((int(latest_snap) - int(earliest_snap))//10 + 1)]

#now now I set up a nested loop to iterate over each a in my a_list in each of my VELA_dir halomass folder
for a in a_list:
    print('Generating dataset for a: %s' % a)
    
    central_ids, satellite_ids = [], []
    
    central_sim, satellite_sim = [], []
    
    gas_mass_central, gas_mass_satellite = [], []
    
    gas_ratio_central, gas_ratio_satellite = [], []
    
    central_all_mass_mvir, satellite_all_mass_mvir = [], []
    
    for halomass_dir in halomass_dirs:
        print('Trying file: %s/halomass%s.ascii' % (halomass_dir, a))
        try:
            readfile = open('%s/halomass%s.ascii' % (halomass_dir, a))
            lines = readfile.readlines()
            halomass_lines = []
            for line in lines:
                halomass_lines.append(line.split())
            del halomass_lines[0]
            for data_lists in halomass_lines:
                #filter out the satellites (pid =/= -1) from the central galaxies they orbit (pid = -1)
                if float(data_lists[1]) == -1:
                    central_ids.append(float(data_lists[0]))
                    
                    central_sim.append(halomass_dir[-2:])
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    
                    gas_mass_central.append(float(data_lists[8]) + float(data_lists[7]))
                    
                    #Mvir = darkmatter + stars + gas @ rvir
                    Mvir = float(data_lists[2]) + float(data_lists[7]) + float(data_lists[8])
                    central_all_mass_mvir.append(Mvir)
                    
                    gas_ratio_central.append((float(data_lists[8]) + float(data_lists[7])) / Mvir)
                    
                else:
                    satellite_ids.append(float(data_lists[0]))
                    
                    satellite_sim.append(halomass_dir[-2:])
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    
                    gas_mass_satellite.append(float(data_lists[8]) + float(data_lists[7]))
                    
                    Mpeak = float(data_lists[3]) + float(data_lists[7]) + float(data_lists[8])
                    satellite_all_mass_mvir.append(Mpeak)
                    
                    gas_ratio_satellite.append((float(data_lists[8]) + float(data_lists[7])) / Mpeak)
                    
            readfile.close()
            
        except FileNotFoundError:
            print('No file: %s/halomass%s.ascii' % (halomass_dir, a))
            
    gas_ratio_central_lists.append(gas_ratio_central)
        
    central_ids_lists.append(central_ids)
    
    central_sim_lists.append(central_sim)
            
   
            

Generating dataset for a: 100
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA08/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA09/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA10/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA11/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA12/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA13/halomass100.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA14/halomass100.ascii
Trying file: /Us

Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA08/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA09/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA10/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA11/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA12/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA13/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA14/halomass250.ascii
Trying file: /Users/user1/documents/GEN6/rocks

Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA08/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA09/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA10/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA11/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA12/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA13/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA14/halomass400.ascii
Trying file: /Users/user1/documents/GEN6/rocks

Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA10/halomass520.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA11/halomass520.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA12/halomass520.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA13/halomass520.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA14/halomass520.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA15/halomass520.ascii
Generating dataset for a: 530
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass530.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass530.ascii
Trying file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass530.ascii
No file: /Users/user

In [25]:
print(a_list[40])

start = gas_ratio_central_lists[40][0]
count = 0

halos_of_interest = []

while count < len(gas_ratio_central_lists[40]):
    if gas_ratio_central_lists[40][count] > 0.167:
        halos_of_interest.append((a_list[40], central_sim_lists[40][count], central_ids_lists[40][count]))
        
    count = count + 1


VELAS_of_interest = np.unique([x[1] for x in halos_of_interest])
print(VELAS_of_interest)

ids_of_interest = np.unique([x[2] for x in halos_of_interest if x[1] == '08'])
print(halos_of_interest)
print([x[2] for x in halos_of_interest if x[1] == '08'])

for VELA_numbers in VELAS_of_interest:
    VELA_dir = '/nobackupp2/sflarkin/GEN6/VELA{}'.format(VELA_numbers)
    input_dir = '/nobackupp2/sflarkin/GEN6/rockstar001analysis/VELA{}/hlists'.format(VELA_numbers)
    print(VELA_dir)
    print(input_dir)

500
['08' '10' '11' '14' '15']
[(500, '08', 758123.0), (500, '08', 755273.0), (500, '08', 755429.0), (500, '08', 757482.0), (500, '08', 758089.0), (500, '08', 758039.0), (500, '08', 758194.0), (500, '08', 757452.0), (500, '08', 755133.0), (500, '08', 754857.0), (500, '08', 755204.0), (500, '10', 646850.0), (500, '10', 646757.0), (500, '10', 646891.0), (500, '10', 646952.0), (500, '10', 646730.0), (500, '11', 427964.0), (500, '11', 427983.0), (500, '14', 346146.0), (500, '14', 345761.0), (500, '14', 345762.0), (500, '15', 237964.0)]
[758123.0, 755273.0, 755429.0, 757482.0, 758089.0, 758039.0, 758194.0, 757452.0, 755133.0, 754857.0, 755204.0]
/nobackupp2/sflarkin/GEN6/VELA08
/nobackupp2/sflarkin/GEN6/rockstar001analysis/VELA08/hlists
/nobackupp2/sflarkin/GEN6/VELA10
/nobackupp2/sflarkin/GEN6/rockstar001analysis/VELA10/hlists
/nobackupp2/sflarkin/GEN6/VELA11
/nobackupp2/sflarkin/GEN6/rockstar001analysis/VELA11/hlists
/nobackupp2/sflarkin/GEN6/VELA14
/nobackupp2/sflarkin/GEN6/rockstar001an

In [53]:
data_of_interest = []

for a in a_list:
    #print('Generating dataset for a: %s' % a)
    
    for halomass_dir in halomass_dirs:
        #print('Trying file: %s/halomass%s.ascii' % (halomass_dir, a))
        try:
            readfile = open('%s/halomass%s.ascii' % (halomass_dir, a))
            lines = readfile.readlines()
            halomass_lines = []
            for line in lines:
                halomass_lines.append(line.split())
            del halomass_lines[0]
            for data_lists in halomass_lines:
                #filter out the satellites (pid =/= -1) from the central galaxies they orbit (pid = -1)
                if float(data_lists[0]) in ids_to_check:
                    pos = [x for x,i in enumerate(ids_to_check) if i==float(data_lists[0])][0]
                    if float(data_lists[0])==ids_to_check[pos] and halomass_dir[-2:]==sim_to_check[pos]:
                        data_of_interest.append(data_lists)
                    
            readfile.close()
            
        except FileNotFoundError:
            print('No file: %s/halomass%s.ascii' % (halomass_dir, a))

No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass410.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass410.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass420.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass420.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass430.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass430.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass440.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA07/halomass440.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA06/halomass450.ascii
No file: /Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA0

In [54]:
print(data_of_interest[0])

['758123', '-1', '14342857142.857143', '14771428571.428572', '232967192.17788848', '288212965.1790138', '327775888.2936169', '375907942.85027605', '4920873621.37857', '41612857636.16234']


In [60]:
for data in data_of_interest:
    print(float(data[8])/10**8)

49.2087362137857
49.68251418807518
36.985966649129544
16.648714546286136
15.402796146382071
11.201562135953026
7.2162094652014135
1.5847572227724285
1.2923162982358019
1.2320047688172806
1.3769468570631367
6.568684947552465
6.681461322690719
7.1622294939781925
2.9840786977588
0.30585287126410116
36.65023806647181
7.844180233180741
11.420907690057227
4.422502246555867
0.29478327003767546
5.143480341356957


In [65]:
print(star_ratio_central)

[0.01533163931548889, 0.009233743260987817, 0.0027866784725366005, 0.01060791837618213, 0.002513658062436639, 0.0016034402786347475, 0.0020753192827016056, 0.004530307836891295, 0.00015160600812546337, 0.0005952394148511919, 0.00011594861567030873, 7.7002054762123e-05, 2.6221164681028073e-05, 0.00014322814252592137, 5.0462133570534935e-05, 6.44615895460659e-05, 0.0001822480254446704, 0.0005601059026672582, 5.7492329374807076e-05, 0.00014162002643692634, 2.5660556859696746e-05, 0.001674655132776732, 1.93862313554358e-05, 9.468832652807796e-05, 0.0001470454957333832, 5.7523728945997115e-05, 9.146641675450223e-05, 0.0005969344075657282, 0.001023332176212694, 0.0, 0.0004010046963632409, 0.0, 3.428415384384165e-05, 0.0, 0.0, 0.0001442812063268006, 6.881609788220522e-05, 2.628820815945018e-05, 0.0, 0.0, 0.001404471236742047, 1.5602758059543202e-05, 0.0, 4.022889750373948e-05, 8.516398534853073e-05, 0.0001983210954519292, 0.0, 9.13228355340418e-05, 0.0, 0.0, 0.00015236500793557967, 0.0, 3.000